In [17]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 핸드 모델 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

# 건반 정의 (x 시작점, y 시작점, 너비, 높이) 및 각 건반에 해당하는 음계
keys = [(100, 350, 50, 100), (150, 350, 50, 100), (200, 350, 50, 100), (250, 350, 50, 100), (300, 350, 50, 100)]
key_scales = ['C', 'D', 'E', 'F', 'G']  # 각 건반에 해당하는 음계
key_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 165, 0)]
key_status = [False] * len(keys)  # 건반의 상태 (눌림/눌리지 않음)

while True:
    success, frame = cap.read()
    if not success:
        break

    # 프레임 좌우 반전
    frame = cv2.flip(frame, 1)  
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    # 건반 그리기
    for i, key in enumerate(keys):
        cv2.rectangle(frame, (key[0], key[1]), (key[0] + key[2], key[1] + key[3]), key_colors[i])
        cv2.putText(frame, key_scales[i], (key[0] + 10, key[1] + 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 인덱스 손가락 끝의 좌표
            x = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * frame.shape[1])
            y = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * frame.shape[0])
            
            # 건반을 눌렀는지 확인하고, 해당하는 음계 출력
            for i, key in enumerate(keys):
                cv2.rectangle(frame, (key[0], key[1]), (key[0] + key[2], key[1] + key[3]), key_colors[i], cv2.FILLED if key_status[i] else 2)
                if key[0] <= x <= key[0] + key[2] and key[1] <= y <= key[1] + key[3]:
                    cv2.putText(frame, key_scales[i], (key[0] + 10, key[1] + 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    key_status[i] = True
                else:
                    key_status[i] = False

    cv2.imshow('Virtual Piano Scales', frame)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
